In [12]:
# I chose the Exponential Smoothing (Holt-Winters) model

# assignment1.py  —— paste this whole file

import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# ---- 1) load train / test data from raw URLs (works in Colab AND GitHub Actions) ----
TRAIN_URL = "https://raw.githubusercontent.com/dustywhite7/econ8310-assignment1/main/assignment_data_train.csv"
TEST_URL  = "https://raw.githubusercontent.com/dustywhite7/econ8310-assignment1/main/assignment_data_test.csv"

train_df = pd.read_csv(TRAIN_URL, parse_dates=["Timestamp"])
test_df  = pd.read_csv(TEST_URL,  parse_dates=["Timestamp"])

# sort + set hourly freq so statsmodels knows the frequency
train_df = train_df.sort_values("Timestamp").set_index("Timestamp").asfreq("H")

# numeric target with simple gap fill
y_train = (
    pd.to_numeric(train_df["trips"], errors="coerce")
    .astype(float)
    .interpolate(limit_direction="both")
)

# ---- 2) build the model object (do NOT call .fit() here) ----
model = ExponentialSmoothing(
    y_train,
    trend="add",
    seasonal="add",
    seasonal_periods=24,
    initialization_method="estimated",
)

# ---- 3) fit the model ----
modelFit = model.fit(optimized=True, use_brute=True)

# ---- 4) forecast exactly len(test) steps and expose as 1-D Python floats ----
h = len(test_df)
_fc = modelFit.forecast(steps=h)

# IMPORTANT: make it a plain Python-list of floats, not numpy.float64
pred = list(map(float, np.asarray(_fc).ravel()))


/tmp/ipython-input-228930099.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  train_df = train_df.sort_values("Timestamp").set_index("Timestamp").asfreq("H")
